In [1]:
#| default_exp small_dataset_creation

In [2]:
#| export
from nbdev.showdoc import *

In [6]:
#| export
from pathlib import Path
import os
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from dataclasses import dataclass, field
from collections import Counter
import shutil

In [7]:
#| export
import timm
from fastai.imports import *
from fastai.vision.all import *

In [8]:
#| export
rnd_seed = 42
n_s = 5
set_seed(rnd_seed)

In [18]:
#| export 
def process_trn_df(df_train):
    cols = ['image_name', 'cancer']
    return (
        df_train
        .assign(image_name=lambda df1: df1['patient_id'].astype(str) + "_"+ df1['image_id'].astype(str) +".png")
        .loc[:,cols]
    )

In [10]:
#| export
@dataclass
class StratifyYSplitData:

    X:str # column name for X
    Y:str # column name for Y which needs to be stratified
    df_t:pd.DataFrame=field(repr=False) # dataframe with all images and context data
    train_path:Path=field(repr=False) # path to train images
    shuffle_d:bool=True # shuffle data or not
    random_state_:int=42 # random state
    test_size:float=0.2 # test size

    def __post_init__(self):
        self.x_trn,  self.x_tst, self.y_trn, self.y_tst = train_test_split(
                                                                         self.df_t[self.X].values,
                                                                         self.df_t[self.Y].values,
                                                                         test_size=self.test_size,
                                                                         stratify=self.df_t[self.Y].values
                                                                         )
    def get_data(
                    self,
                    )->Tuple[List[str], List[int], List[str], List[int]]:
        trn_img = [self.train_path/str(i) for i in self.x_trn]
        trn_lbl = self.y_trn
        val_img = [self.train_path/str(i) for i in self.x_tst]
        val_lbl = self.y_tst

        return (trn_img, trn_lbl), (val_img, val_lbl)

    def __getitem__(self, idx:int)->Tuple[List[str], List[int]]:
        if idx == 0:
            (trn_img, trn_lbl), (_, _) = self.get_data()
            return trn_img, trn_lbl
        else:
            (_, _), (val_img, val_lbl) = self.get_data()
            return val_img, val_lbl
    
    def get_label_dict(
                       self,
                       data_list:str,
                       image_list:List[Path]
                       )->Dict[int, str]:
        
        if data_list == 'train':
            x, y = self[0]
            #print(x)
            return {i:j for i,j in zip(x,y)}

        elif data_list == 'valid':
            x, y = self[1]
            return {i:j for i,j in zip(x, y)}

        else:
            root_path = image_list[0].parent
            im_name_list = [Path(i).name for i in image_list]

            x,y = self[0]
            actual_dic =  {i.name:j for i,j in zip(x, y) if Path(i).name in im_name_list}
            return {k:v for k,v in actual_dic.items()}

In [11]:
#| export
def create_small_dataset(
                        #source_:Path, # Present image path
                        destination_:Path, # new image path where small dataset will be created
                        image_list:List[Path], # list of images to be copied
                        symlink_:bool=False, # whether to create symlink or copy
                        )->None:
    """
    Creates a small dataset for faster training
    """
    destination_.mkdir(exist_ok=True, parents=True)


    if symlink_:
        print(f"Creating symbolic link of {len(image_list)} images in {destination_}")

        [Path(f'{destination_}/{i.name}').symlink_to(i) for i in image_list if not (destination_/i.name).exists()]

        print(f' number of files found in {destination_} == {len(destination_.ls())}')
    else:
        print(f"Copying ... {len(image_list)} images in {destination_}")
        [shutil.copyfile(i, f'{destination_}/{i.name}') for i in image_list]
        print(f' number of files found in {destination_} == {len(destination_.ls())}')


In [20]:
#| hide
import nbdev; nbdev.nbdev_export('01_small_dataset_creation.ipynb')